In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import STL
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks, peak_prominences, peak_widths
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from joblib import Parallel, delayed

from TSB_AD.utils.slidingWindows import argrelextrema
from TSB_AD.utils.slidingWindows import find_length_rank


def load_series(files):
    results = {}
    for f in files:
        if not f.endswith(".csv"):
            f = f + ".csv"
        df = pd.read_csv("benchmark_exp/TSB-AD/TSB-AD-U/" + f)
        results[f.split(".")[0]] = df
    return results

def find_length(data):
    a, b = np.quantile(data, [0.001, 0.999])
    data = np.clip(data, a, b)
    
    #data = data[:min(20000, len(data))]
    n_lags = 5000
    base = 3
    auto_corr = acf(data, nlags=n_lags, fft=True)[base:]
    try:
        peaks, _ = find_peaks(auto_corr)
        prominences = peak_prominences(auto_corr, peaks)[0]
        peaks = peaks + base
        prominent_peak_idx = np.argmax(prominences)
    except:
        return []
    if prominences[prominent_peak_idx] < 0.1:
        result = []
    else:
        highest_peak = np.argmax(auto_corr[peaks - base])
        ac_of_prominent = auto_corr[peaks[prominent_peak_idx] - base]
        # 99% significance level of autocorrelation
        if ac_of_prominent < 2.576 / np.sqrt(len(data)):
            return []
        result = [peaks[prominent_peak_idx]]
        if highest_peak != prominent_peak_idx and prominences[highest_peak] > 0.1:
            result.append(peaks[highest_peak])
    return result

In [ ]:
val_data = load_series(pd.read_csv("benchmark_exp/TSB-AD/File-List/TSB-AD-U-Eval-List.csv", header=0)['file_name'])

In [ ]:
len(val_data)

In [ ]:
def iqr(s):
    return s.quantile([0.75]).iloc[0] - s.quantile([0.25]).iloc[0]

In [ ]:
def get_stl_score(name, series):
    mine = find_length(series)
    if not len(mine):
        return name, None
    mine = mine[0]
    fit = STL(series, period=mine).fit()
    residual_r2_score = 1 - (fit.resid ** 2).mean() / series.var()
    
    obs = np.maximum(iqr(fit.observed), 1e-5)
    sea = iqr(fit.seasonal)
    trend = iqr(fit.trend)
    resid = iqr(fit.resid)
    return name, {'r2': residual_r2_score, 'season_iqr': sea / obs, 'trend_iqr': trend / obs, 'res_iqr': resid / obs}

In [ ]:
scores = Parallel(n_jobs=-1)(delayed(get_stl_score)(name, s['Data']) for name, s in val_data.items())

In [ ]:
scores_dict = {a[0]:a[1] for a in scores if a is not None}

In [ ]:
lengths_list = []
for name, s in val_data.items():
    data = s['Data']
    n_lags = 5000
    autocorr = acf(data, nlags=n_lags, fft=True)
    mine = find_length(data)
    if len(mine) == 2:
        mine, second = mine
    elif len(mine):
        mine, second = mine[0], None
    else:
        mine, second = None, None
    theirs = find_length_rank(data)
    length = len(data)

    if mine is not None:
        mine_sig = autocorr[mine] > 2.576 / np.sqrt(length)
    else:
        mine_sig = False
    theirs_sig = autocorr[theirs] > 2.576 / np.sqrt(length)

    this_res = {'name': name, 'mine': mine, 'second': second, 'theirs': theirs, 'length': len(data), 'mine_sig': mine_sig, 'theirs_sig': theirs_sig}
    if mine is not None:
        this_res.update(scores_dict[name])
    lengths_list.append(this_res)

In [ ]:
lengths = pd.DataFrame(lengths_list)
lengths

In [ ]:
lengths.to_csv("series_length_debugging_iqr.csv")

In [ ]:
lengths['score'] = np.maximum(0, lengths['score'])

In [ ]:
(lengths.score < .5).sum()

In [ ]:
lengths.score.hist(bins=30)

In [ ]:
weird = lengths[~lengths.mine.isna() & ~lengths['mine_sig']]
weird

In [ ]:
interesting = lengths[lengths['score'] < 0.5]
interesting[::5][:30]

In [ ]:
gah = ['025_UCR_Anomaly_CIMIS44AirTemperature1_2046_5391',
       '029_UCR_Anomaly_ECG2_7500_16000',
       '033_UCR_Anomaly_Italianpowerdemand_7482_29480',
       '036_UCR_Anomaly_insectEPG3_2499_7000',
       '051_UCR_Anomaly_DISTORTEDresperation4_48812_128430',
       '080_UCR_Anomaly_gait1_16250_38500',
       '085_UCR_Anomaly_Italianpowerdemand_29895_39240',
       '107_UCR_Anomaly_gait3_16250_59900',
       '154_UCR_Anomaly_resperation11_48750_110800',
       '208_UCR_Anomaly_DISTORTEDresperation1_50000_110260',
       '216_UCR_Anomaly_resperation10_48750_130700',
       '228_UCR_Anomaly_taichidbS0715Master_50000_884100',
       '504_IOPS_KPI-8723f0fb-eaef-32e6-b372-6034c9c04b80_5578_5678',
       '506_IOPS_KPI-6d1114ae-be04-3c46-b5aa-be1a003a57cd_8620_8720',
       '507_IOPS_KPI-301c70d8-1630-35ac-8f96-bc1b6f4359ea_1483_1583',
       '508_IOPS_KPI-e0747cad-8dc8-38a9-a9ab-855b61f5551d_500_341',
       '509_IOPS_KPI-0efb375b-b902-3661-ab23-9a0bb799f4e3_500_264',
       '515_IOPS_KPI-ab216663-dcc2-3a24-b1ee-2c3e550e06c9_500_77',
       '517_IOPS_KPI-8723f0fb-eaef-32e6-b372-6034c9c04b80_5638_5738',
       '530_WSD_150_4521_4599', '532_WSD_171_643_743', '533_WSD_168_4521_6454',
       '534_WSD_187_4559_12588', '538_WSD_153_500_276',
       '540_WSD_104_2411_2511', '544_WSD_191_4559_9714', '545_WSD_37_500_78',
       '548_WSD_57_4521_6828', '549_WSD_154_1129_1229', '554_WSD_184_500_53']

In [ ]:
for _, (n, m, snd, t, l, ms, ts, score) in list(lengths.iterrows()):
    if n not in gah:
        continue
    plt.figure(figsize=(20, 5))
    plt.title(f"{n} mine: {m} ({ms}) second: {snd} theirs: {t} ({ts}) score {score}")
    s = val_data[n]['Data']
    if not np.isnan(m):
        m = int(m)
        plt.vlines(np.arange(0, len(s), int(m)), label="mine", ymin=s.min(), ymax=s.max(), alpha=0.5, color='green')
    if not np.isnan(snd):
        plt.vlines(np.arange(0, len(s), int(snd)), label="mine", ymin=s.min(), ymax=s.max(), alpha=0.5, color='green')
    # plt.vlines(np.arange(0, len(s), t), label="theirs", ymin=s.min(), ymax=s.max(), alpha=0.5, color='red')
    s.plot()
    plt.figure(figsize=(20, 5))
    n_lags = min(5000, 20 * max(m, t))
    autocorr = acf(s, nlags=n_lags, fft=True)
    plt.plot(autocorr)
    peaks, _ = find_peaks(autocorr)
    if len(peaks):
        prominences = peak_prominences(autocorr, peaks)[0]
        widths = peak_widths(autocorr, peaks)[0]
        for i, p in enumerate(peaks[:10]):
            plt.text(p, autocorr[p], f"p: {prominences[i]:.2f} w: {widths[i]:.2f}")
        prominent_peak_idx = np.argmax(prominences)
        # msg = ""
        # if widths[prominent_peak_idx] > peaks[prominent_peak_idx]:
        #     msg = msg + " early"
        # if prominences[prominent_peak_idx] < 0.1:
        #     msg = msg + " too low"
        # plt.title(msg)
        plt.text(peaks[prominent_peak_idx], autocorr[peaks[prominent_peak_idx]], f"p: {prominences[prominent_peak_idx]:.2f} w: {widths[prominent_peak_idx]:.2f}")
        plt.plot(peaks, autocorr[peaks], 'o', c='k')
    plt.hlines([2.576 / np.sqrt(len(s))], xmin=0, xmax=len(autocorr))
    if not np.isnan(m):
        plt.plot([m], autocorr[m], 'o', c='g')
    plt.plot([t], autocorr[t], 'o', c='r')